In [1]:
from selenium import webdriver as wd
import chromedriver_binary
my_phone_number="<enter-your-number>"

In [61]:
wd = wd.Chrome()

In [62]:
wd.implicitly_wait(10)

In [63]:
wd.get("https://amazon.com")

In [64]:
import time
time.sleep(5)

wd.find_element_by_id("nav-global-location-popover-link").click()
postcode_field = wd.find_element_by_id("GLUXZipUpdateInput")
postcode_field.send_keys("80021")
time.sleep(2)
wd.find_element_by_id("GLUXZipUpdate").click()
close_element = wd.find_element_by_xpath("/html/body/div[5]/div/div/div[2]/span/span/input")
close_element.click()

In [18]:
wd.get("https://www.amazon.com/s?k=rtx+3080+ti&i=computers&bbn=284822&rh=n%3A284822%2Cp_n_feature_twenty-one_browse-bin%3A21563385011%2Cp_n_feature_twenty_browse-bin%3A23572110011&dc&qid=1630186292&rnid=2661599011&ref=sr_nr_p_n_availability_2")

In [19]:
from bs4 import BeautifulSoup

In [20]:
soup = BeautifulSoup(wd.page_source)

In [21]:
results = soup.findAll("div", {"data-component-type": "s-search-result"})
for result in results:
    title = result.find("span", {"class": "a-text-normal"})
    sponsored = result.find("span", {"class": "a-color-base"}, text="Sponsored")
    price = result.find("span", {"class": "a-price-whole"})
    if not sponsored and price:
        print(title.text)
        print(price.text)
        

ASUS TUF Gaming NVIDIA GeForce RTX 3080 Ti OC Edition Graphics Card (PCIe 4.0, 12GB GDDR6X, HDMI 2.1, DisplayPort 1.4a, Dual Ball Fan Bearings, Military-Grade Certification, GPU Tweak II)
1,899.
ASUS ROG Strix NVIDIA GeForce RTX 3080 Ti OC Edition Gaming Graphics Card (PCIe 4.0, 12GB GDDR6X, HDMI 2.1, DisplayPort 1.4a, Axial-tech Fan Design, 2.9-Slot, Super Alloy Power II, GPU Tweak II)
2,499.
MSI Gaming GeForce RTX 3080 Ti OC Gaming Trio X Graphics Card, 12GB GDRR6X 320-Bit PCIe 4.0, Torx Fan 4.0 Ampere Architecture, DisplayPort x 3 (v1.4) HDMI x 1 w/ Mytrix_HDMI Cable
2,399.
MSI Gaming GeForce RTX 3080 Ti OC Gaming Trio X Graphics Card, 12GB GDRR6X 320-Bit PCIe 4.0, Torx Fan 4.0 Ampere Architecture, DisplayPort x3 (v1.4) HDMI x1 w/ Mytrix_HDMI 2.1 Cable(4k@120Hz/8K@60Hz)
2,399.
PNY GeForce RTX 3080 Ti 12GB XLR8 Gaming Revel Epic-X RGB Triple Fan Graphics Card
2,144.
ZOTAC Gaming GeForce RTX 3080 Ti AMP Holo 12GB GDDR6X 384-bit 19 Gbps PCIE 4.0 Gaming Graphics Card, HoloBlack, IceStor

In [22]:
def getPandaDfForSearchResults(searchResultsPage):
    rows=[]
    for result in searchResultsPage:
        title = result.find("span", {"class": "a-text-normal"})
        sponsored = result.find("span", {"class": "a-color-base"}, text="Sponsored")
        price = result.find("span", {"class": "a-price-whole"})
        url = result.find("a", {"class": "a-link-normal"})
        if price:
            row= [title.text, bool(sponsored), price.text, "https://amazon.com/" + url['href']]
            rows.append(row)
    
    df=pd.DataFrame.from_records(rows, columns=["Title", "Sponsored", "Price", "URL"])
    return df

In [28]:
import pandas as pd

nextPageCssClasses=[]
nextPageLink = wd.find_element_by_class_name("a-last")


df_all_search_results = pd.DataFrame(columns=["Title", "Sponsored", "Price"])
while not "a-disabled" in nextPageCssClasses:
    soup = BeautifulSoup(wd.page_source)
    searchResultsPage = soup.findAll("div", {"data-component-type": "s-search-result"})
    df = getPandaDfForSearchResults(searchResultsPage)
    df_all_search_results = pd.concat([df_all_search_results, df])
                              
    # go to next page
    nextPageLink.click()
    nextPageLink = wd.find_element_by_class_name("a-last")
    nextPageCssClasses = nextPageLink.get_attribute("class").split()
    time.sleep(2)

if "a-disabled" in nextPageCssClasses:
    print("Reached last page of search results")

df_all_search_results    
                            

Reached last page of search results


,Title,Sponsored,Price,URL
0,MSI Gaming GeForce RTX 3080 Ti OC Gaming Trio ...,True,"2,399.",https://amazon.com//gp/slredirect/picassoRedir...
1,MSI Gaming GeForce RTX 3080 Ti OC Gaming Trio ...,True,"2,399.",https://amazon.com//gp/slredirect/picassoRedir...
2,ASUS TUF Gaming NVIDIA GeForce RTX 3080 Ti OC ...,False,"1,899.",https://amazon.com//ASUS-Graphics-DisplayPort-...
3,ASUS ROG Strix NVIDIA GeForce RTX 3080 Ti OC E...,False,"2,499.",https://amazon.com//ASUS-Graphics-DisplayPort-...
4,MSI Gaming GeForce RTX 3080 Ti OC Gaming Trio ...,False,"2,399.",https://amazon.com//GeForce-Graphics-Architect...
5,MSI Gaming GeForce RTX 3080 Ti OC Gaming Trio ...,False,"2,399.",https://amazon.com//GeForce-Graphics-Architect...
6,MSI Gaming GeForce RTX 3080 OC Ventus 3X LHR G...,True,"2,149.",https://amazon.com//gp/slredirect/picassoRedir...
7,PNY GeForce RTX 3080 Ti 12GB XLR8 Gaming Revel...,False,"2,144.",https://amazon.com//PNY-GeForce-Gaming-Epic-X-...
8,MSI Gaming GeForce RTX 3080 OC Ventus 3X LHR G...,True,"2,149.",https://amazon.com//gp/slredirect/picassoRedir...
9,ZOTAC Gaming GeForce RTX 3080 Ti AMP Holo 12GB...,False,"2,130.",https://amazon.com//ZOTAC-Graphics-HoloBlack-I...


In [29]:
df_all_search_results.Title.str.contains("3080 Ti")

0      True
1      True
2      True
3      True
4      True
5      True
6     False
7      True
8     False
9      True
10    False
11    False
12    False
13    False
14    False
15    False
16    False
0      True
1     False
2      True
3      True
4     False
5      True
6     False
7     False
8      True
9     False
0     False
1     False
2     False
3     False
4     False
5     False
6      True
7     False
8     False
9     False
10    False
11    False
12     True
13    False
Name: Title, dtype: bool

In [30]:
df_all_search_results[df_all_search_results.Title.str.contains("3080 Ti")]

,Title,Sponsored,Price,URL
0,MSI Gaming GeForce RTX 3080 Ti OC Gaming Trio ...,True,"2,399.",https://amazon.com//gp/slredirect/picassoRedir...
1,MSI Gaming GeForce RTX 3080 Ti OC Gaming Trio ...,True,"2,399.",https://amazon.com//gp/slredirect/picassoRedir...
2,ASUS TUF Gaming NVIDIA GeForce RTX 3080 Ti OC ...,False,"1,899.",https://amazon.com//ASUS-Graphics-DisplayPort-...
3,ASUS ROG Strix NVIDIA GeForce RTX 3080 Ti OC E...,False,"2,499.",https://amazon.com//ASUS-Graphics-DisplayPort-...
4,MSI Gaming GeForce RTX 3080 Ti OC Gaming Trio ...,False,"2,399.",https://amazon.com//GeForce-Graphics-Architect...
5,MSI Gaming GeForce RTX 3080 Ti OC Gaming Trio ...,False,"2,399.",https://amazon.com//GeForce-Graphics-Architect...
7,PNY GeForce RTX 3080 Ti 12GB XLR8 Gaming Revel...,False,"2,144.",https://amazon.com//PNY-GeForce-Gaming-Epic-X-...
9,ZOTAC Gaming GeForce RTX 3080 Ti AMP Holo 12GB...,False,"2,130.",https://amazon.com//ZOTAC-Graphics-HoloBlack-I...
0,SkyTech Prism II Gaming Computer PC Desktop - ...,True,"3,243.",https://amazon.com//gp/slredirect/picassoRedir...
2,EVGA GeForce RTX 3080 Ti XC3 Ultra Hybrid Gami...,False,"2,225.",https://amazon.com//EVGA-GeForce-Hybrid-12G-P5...


In [31]:
(df_all_search_results.Title.str.contains("3080 Ti")) & (df_all_search_results.Price>= 2000)

TypeError: '>=' not supported between instances of 'str' and 'int'

In [32]:
df_all_search_results.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Title      41 non-null     object
 1   Sponsored  41 non-null     object
 2   Price      41 non-null     object
 3   URL        41 non-null     object
dtypes: object(4)
memory usage: 1.6+ KB


In [33]:
df_all_search_results["Price"] == "2,399."

0      True
1      True
2     False
3     False
4      True
5      True
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
Name: Price, dtype: bool

In [34]:
df_all_search_results['Price'].astype(float)

ValueError: could not convert string to float: '2,399.'

In [35]:
df_all_search_results['Price'].str.replace(",","")

0     2399.
1     2399.
2     1899.
3     2499.
4     2399.
5     2399.
6     2149.
7     2144.
8     2149.
9     2130.
10    4988.
11    1549.
12    2199.
13    1619.
14    2149.
15     767.
16    1418.
0     3243.
1     1549.
2     2225.
3     2444.
4     1599.
5     2412.
6      749.
7     1549.
8     2408.
9     1099.
0     2499.
1     1699.
2     1599.
3     1743.
4     1539.
5     1249.
6     2694.
7     1099.
8     1949.
9     1999.
10    1199.
11    1099.
12    7410.
13    1589.
Name: Price, dtype: object

In [36]:
df_all_search_results['Price'] = df_all_search_results['Price'].str.replace(",","")

In [37]:
df_all_search_results

,Title,Sponsored,Price,URL
0,MSI Gaming GeForce RTX 3080 Ti OC Gaming Trio ...,True,2399.,https://amazon.com//gp/slredirect/picassoRedir...
1,MSI Gaming GeForce RTX 3080 Ti OC Gaming Trio ...,True,2399.,https://amazon.com//gp/slredirect/picassoRedir...
2,ASUS TUF Gaming NVIDIA GeForce RTX 3080 Ti OC ...,False,1899.,https://amazon.com//ASUS-Graphics-DisplayPort-...
3,ASUS ROG Strix NVIDIA GeForce RTX 3080 Ti OC E...,False,2499.,https://amazon.com//ASUS-Graphics-DisplayPort-...
4,MSI Gaming GeForce RTX 3080 Ti OC Gaming Trio ...,False,2399.,https://amazon.com//GeForce-Graphics-Architect...
5,MSI Gaming GeForce RTX 3080 Ti OC Gaming Trio ...,False,2399.,https://amazon.com//GeForce-Graphics-Architect...
6,MSI Gaming GeForce RTX 3080 OC Ventus 3X LHR G...,True,2149.,https://amazon.com//gp/slredirect/picassoRedir...
7,PNY GeForce RTX 3080 Ti 12GB XLR8 Gaming Revel...,False,2144.,https://amazon.com//PNY-GeForce-Gaming-Epic-X-...
8,MSI Gaming GeForce RTX 3080 OC Ventus 3X LHR G...,True,2149.,https://amazon.com//gp/slredirect/picassoRedir...
9,ZOTAC Gaming GeForce RTX 3080 Ti AMP Holo 12GB...,False,2130.,https://amazon.com//ZOTAC-Graphics-HoloBlack-I...


In [38]:
df_all_search_results['Price'].astype(float)

0     2399.0
1     2399.0
2     1899.0
3     2499.0
4     2399.0
5     2399.0
6     2149.0
7     2144.0
8     2149.0
9     2130.0
10    4988.0
11    1549.0
12    2199.0
13    1619.0
14    2149.0
15     767.0
16    1418.0
0     3243.0
1     1549.0
2     2225.0
3     2444.0
4     1599.0
5     2412.0
6      749.0
7     1549.0
8     2408.0
9     1099.0
0     2499.0
1     1699.0
2     1599.0
3     1743.0
4     1539.0
5     1249.0
6     2694.0
7     1099.0
8     1949.0
9     1999.0
10    1199.0
11    1099.0
12    7410.0
13    1589.0
Name: Price, dtype: float64

In [39]:
df_all_search_results['Price'] = df_all_search_results['Price'].astype(float)

In [40]:
df_all_search_results

,Title,Sponsored,Price,URL
0,MSI Gaming GeForce RTX 3080 Ti OC Gaming Trio ...,True,2399.0,https://amazon.com//gp/slredirect/picassoRedir...
1,MSI Gaming GeForce RTX 3080 Ti OC Gaming Trio ...,True,2399.0,https://amazon.com//gp/slredirect/picassoRedir...
2,ASUS TUF Gaming NVIDIA GeForce RTX 3080 Ti OC ...,False,1899.0,https://amazon.com//ASUS-Graphics-DisplayPort-...
3,ASUS ROG Strix NVIDIA GeForce RTX 3080 Ti OC E...,False,2499.0,https://amazon.com//ASUS-Graphics-DisplayPort-...
4,MSI Gaming GeForce RTX 3080 Ti OC Gaming Trio ...,False,2399.0,https://amazon.com//GeForce-Graphics-Architect...
5,MSI Gaming GeForce RTX 3080 Ti OC Gaming Trio ...,False,2399.0,https://amazon.com//GeForce-Graphics-Architect...
6,MSI Gaming GeForce RTX 3080 OC Ventus 3X LHR G...,True,2149.0,https://amazon.com//gp/slredirect/picassoRedir...
7,PNY GeForce RTX 3080 Ti 12GB XLR8 Gaming Revel...,False,2144.0,https://amazon.com//PNY-GeForce-Gaming-Epic-X-...
8,MSI Gaming GeForce RTX 3080 OC Ventus 3X LHR G...,True,2149.0,https://amazon.com//gp/slredirect/picassoRedir...
9,ZOTAC Gaming GeForce RTX 3080 Ti AMP Holo 12GB...,False,2130.0,https://amazon.com//ZOTAC-Graphics-HoloBlack-I...


In [41]:
df_all_search_results.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41 entries, 0 to 13
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Title      41 non-null     object 
 1   Sponsored  41 non-null     object 
 2   Price      41 non-null     float64
 3   URL        41 non-null     object 
dtypes: float64(1), object(3)
memory usage: 1.6+ KB


In [42]:
(df_all_search_results.Title.str.contains("3080 Ti")) & (df_all_search_results.Price <= 2200)

0     False
1     False
2      True
3     False
4     False
5     False
6     False
7      True
8     False
9      True
10    False
11    False
12    False
13    False
14    False
15    False
16    False
0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
dtype: bool

In [43]:
df_all_search_results = df_all_search_results[(df_all_search_results.Title.str.contains("3080 Ti")) & (df_all_search_results.Price <= 1500)]

In [44]:
df_all_search_results

,Title,Sponsored,Price,URL


In [47]:
import pandas as pd



def getSearchResultsForPage(url):
    wd.get(url)
    nextPageCssClasses=[]
    nextPageLink = wd.find_element_by_class_name("a-last")


    df_all_search_results = pd.DataFrame(columns=["Title", "Sponsored", "Price"])
    while not "a-disabled" in nextPageCssClasses:    
        soup = BeautifulSoup(wd.page_source)
        searchResultsPage = soup.findAll("div", {"data-component-type": "s-search-result"})
        df = getPandaDfForSearchResults(searchResultsPage)
        df_all_search_results = pd.concat([df_all_search_results, df])

        # go to next page
        nextPageLink.click()
        nextPageLink = wd.find_element_by_class_name("a-last")
        nextPageCssClasses = nextPageLink.get_attribute("class").split()
        time.sleep(2)

    if "a-disabled" in nextPageCssClasses:
        print("Reached last page of search results")

    df_all_search_results['Price'] = df_all_search_results['Price'].str.replace(",","")
    df_all_search_results['Price'] = df_all_search_results['Price'].astype(float)
    return df_all_search_results    
                            

In [48]:
getSearchResultsForPage("https://www.amazon.com/s?k=rtx+3080+ti&i=computers&bbn=284822&rh=n%3A284822%2Cp_n_feature_twenty-one_browse-bin%3A21563385011%2Cp_n_feature_twenty_browse-bin%3A23572110011&dc&qid=1630186292&rnid=2661599011&ref=sr_nr_p_n_availability_2")

Reached last page of search results


,Title,Sponsored,Price,URL
0,MSI Gaming GeForce RTX 3080 Ti OC Gaming Trio ...,True,2399.0,https://amazon.com//gp/slredirect/picassoRedir...
1,MSI Gaming GeForce RTX 3080 Ti OC Gaming Trio ...,True,2399.0,https://amazon.com//gp/slredirect/picassoRedir...
2,MSI Gaming GeForce RTX 3080 Ti 12GB GDRR6X 320...,False,1749.0,https://amazon.com//MSI-Gaming-RTX-3080-Trio/d...
3,ASUS TUF Gaming NVIDIA GeForce RTX 3080 Ti OC ...,False,1899.0,https://amazon.com//ASUS-Graphics-DisplayPort-...
4,ASUS ROG Strix NVIDIA GeForce RTX 3080 Ti OC E...,False,2499.0,https://amazon.com//ASUS-Graphics-DisplayPort-...
5,MSI Gaming GeForce RTX 3080 Ti OC Gaming Trio ...,False,2399.0,https://amazon.com//GeForce-Graphics-Architect...
6,MSI Gaming GeForce RTX 3080 Ti OC Gaming Trio ...,False,2399.0,https://amazon.com//GeForce-Graphics-Architect...
7,PNY NVIDIA Quadro RTX A6000 48GB GDDR6 Graphic...,True,4988.0,https://amazon.com//gp/slredirect/picassoRedir...
8,PNY GeForce RTX 3080 Ti 12GB XLR8 Gaming Revel...,False,2144.0,https://amazon.com//PNY-GeForce-Gaming-Epic-X-...
9,MSI Gaming GeForce RTX 3080 OC Ventus 3X LHR G...,True,2149.0,https://amazon.com//gp/slredirect/picassoRedir...


In [70]:
import pywhatkit as kit
def sendWhatsAppMessage(df):
    for index, graphicsCardRow in df.iterrows():
        print(graphicsCardRow.URL)
        wd.get(graphicsCardRow.URL)
        screenshotFilepath="C:\\Users\\armin\\Downloads\\screenshot_{index}.png"
        wd.save_screenshot(screenshotFilepath)
        kit.sendwhats_image(my_phone_number, screenshotFilepath, graphicsCardRow.URL, tab_close=True)
       
        
        

    

In [71]:
while (True):
    df = getSearchResultsForPage("https://www.amazon.com/s?k=rtx+3080+ti&i=computers&bbn=284822&rh=n%3A284822%2Cp_n_feature_twenty-one_browse-bin%3A21563385011%2Cp_n_feature_twenty_browse-bin%3A23572110011&dc&qid=1630186292&rnid=2661599011&ref=sr_nr_p_n_availability_2")
    
    df_query = df[(df.Title.str.contains("3080 Ti")) & (df.Price <= 2200)]
    
    if (not df_query.empty):
        print("Found products that I want to buy")
        sendWhatsAppMessage(df_query)
        print(df_query)
        break
    else:
     time.sleep(60)
    

Reached last page of search results
Found products that I want to buy
https://amazon.com//ASUS-Graphics-DisplayPort-Military-Grade-Certification/dp/B096L3GLYS/ref=sr_1_5?dchild=1&keywords=rtx+3080+ti&qid=1631456155&refinements=p_n_feature_twenty-one_browse-bin%3A21563385011%2Cp_n_feature_twenty_browse-bin%3A23572110011&rnid=2661599011&s=pc&sr=1-5
https://amazon.com//PNY-GeForce-Gaming-Epic-X-Graphics/dp/B0968PSVPH/ref=sr_1_9?dchild=1&keywords=rtx+3080+ti&qid=1631456155&refinements=p_n_feature_twenty-one_browse-bin%3A21563385011%2Cp_n_feature_twenty_browse-bin%3A23572110011&rnid=2661599011&s=pc&sr=1-9
https://amazon.com//ZOTAC-Graphics-HoloBlack-IceStorm-ZT-A30810F-10P/dp/B08V1ZWGVR/ref=sr_1_20?dchild=1&keywords=rtx+3080+ti&qid=1631456155&refinements=p_n_feature_twenty-one_browse-bin%3A21563385011%2Cp_n_feature_twenty_browse-bin%3A23572110011&rnid=2661599011&s=pc&sr=1-20
                                               Title Sponsored   Price  \
2  ASUS TUF Gaming NVIDIA GeForce RTX 3080 

In [ ]:
import pywhatkit as kit